In [5]:
import cv2
import numpy as np
import yaml
import os

class ArUcoMarkerDetector:
    def __init__(self, calibration_file='calibration.yaml', camera_index=0):
        # Load calibration data
        with open(calibration_file, 'r') as f:
            calibration_data = yaml.safe_load(f)
        self.mtx = np.array(calibration_data['camera_matrix'])
        self.dist = np.array(calibration_data['dist_coeff'])

        # Initialize video capture
        self.cap = cv2.VideoCapture(camera_index)
        self.aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_6X6_250)
        self.parameters = cv2.aruco.DetectorParameters()

        # Get optimal camera matrix
        ret, img = self.cap.read()
        if not ret:
            raise ValueError("Error: Could not read from the camera.")
        h, w = img.shape[:2]
        self.newcameramtx, self.roi = cv2.getOptimalNewCameraMatrix(self.mtx, self.dist, (w, h), 1, (w, h))

    def detect_markers(self):
        while True:
            ret, frame = self.cap.read()
            if not ret:
                print("Error: Could not read frame.")
                break

            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            corners, ids, _ = cv2.aruco.detectMarkers(gray, self.aruco_dict, parameters=self.parameters)

            start_point, end_point = None, None

            if ids is not None:
                start_point, end_point = self._process_markers(frame, corners, ids)
                self._process_rectangle(frame, corners, ids, start_point, end_point)

            cv2.imshow('ArUco Marker Detection', frame)

            # Exit on 'q'
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        self.cap.release()
        cv2.destroyAllWindows()

    def _process_markers(self, frame, corners, ids):
        start_point, end_point = None, None
        for i, marker_id in enumerate(ids.flatten()):
            marker_center = np.mean(corners[i][0], axis=0)

            if marker_id == 6:  # START
                start_point = marker_center
                cv2.circle(frame, tuple(start_point.astype(int)), 5, (0, 255, 0), -1)

            elif marker_id == 5:  # END
                end_point = marker_center
                cv2.circle(frame, tuple(end_point.astype(int)), 5, (0, 0, 255), -1)
        return start_point, end_point

    def _process_rectangle(self, frame, corners, ids, start_point, end_point):
        required_ids = {1, 2, 3, 4}
        marker_positions = {marker_id: corners[i][0] for i, marker_id in enumerate(ids.flatten())}

        if required_ids.issubset(marker_positions.keys()):
            rect_corners = np.array([
                marker_positions[1][0],
                marker_positions[2][1],
                marker_positions[3][2],
                marker_positions[4][3],
            ], dtype=np.float32)

            if rect_corners.shape == (4, 2):
                cv2.polylines(frame, [np.int32(rect_corners)], isClosed=True, color=(0, 0, 255), thickness=3)
                self._warp_perspective(frame, rect_corners, start_point, end_point)

    def _warp_perspective(self, frame, rect_corners, start_point, end_point):
        width, height = 600, 800
        dst_corners = np.array([
            [0, 0],
            [width - 1, 0],
            [width - 1, height - 1],
            [0, height - 1]
        ], dtype=np.float32)

        matrix = cv2.getPerspectiveTransform(rect_corners, dst_corners)
        warped_image = cv2.warpPerspective(frame, matrix, (width, height))

        gray_warped = cv2.cvtColor(warped_image, cv2.COLOR_BGR2GRAY)
        _, binary_map = cv2.threshold(gray_warped, 100, 255, cv2.THRESH_BINARY_INV)

        circle_padding = 100
        for i, marker_id in enumerate(ids.flatten()):
            if marker_id in {5, 6}:
                marker_center = np.mean(corners[i][0], axis=0)
                warped_center = cv2.perspectiveTransform(np.array([[marker_center]], dtype=np.float32), matrix)[0][0]
                warped_center_int = tuple(np.clip(warped_center.astype(int), 0, [binary_map.shape[1] - 1, binary_map.shape[0] - 1]))
                cv2.circle(binary_map, warped_center_int, circle_padding, 0, -1)

        grid_map = np.zeros_like(binary_map, dtype=int)
        grid_map[binary_map == 255] = 1

        if start_point is not None:
            start_warped = cv2.perspectiveTransform(np.array([[start_point]], dtype=np.float32), matrix)[0][0]
            start_coords = tuple(start_warped.astype(int))
            grid_map[start_coords[1], start_coords[0]] = 3

        if end_point is not None:
            end_warped = cv2.perspectiveTransform(np.array([[end_point]], dtype=np.float32), matrix)[0][0]
            end_coords = tuple(end_warped.astype(int))
            grid_map[end_coords[1], end_coords[0]] = 2

        cv2.imshow('Warped Image', warped_image)
        color_map = np.zeros((*binary_map.shape, 3), dtype=np.uint8)
        color_map[binary_map == 0] = (255, 255, 255)
        color_map[binary_map == 255] = (0, 0, 0)
        color_map[grid_map == 2] = (0, 0, 255)
        color_map[grid_map == 3] = (0, 255, 0)
        cv2.imshow('Binary Map', color_map)

        self._save_maps(grid_map)

    def _save_maps(self, grid_map):
        current_dir = os.getcwd()
        folder_name = os.path.join(current_dir, "Maps")
        if not os.path.exists(folder_name):
            os.makedirs(folder_name)
        np.savetxt(os.path.join(folder_name, "grid_map_6_8.txt"), grid_map, fmt='%d', delimiter=' ', comments='')

if __name__ == "__main__":
    detector = ArUcoMarkerDetector()
    detector.detect_markers()


AttributeError: module 'cv2.aruco' has no attribute 'detectMarkers'